<a href="https://colab.research.google.com/github/IanWangg/DSFPG/blob/master/DSFPG_Double_Q.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

%cd gdrive/My Drive/Workplace

# !pip install -e ./pybullet-gym

import sys
sys.path.append('./pybullet-gym')
sys.path.append('./pybullet')
import pybulletgym

Mounted at /content/gdrive
/content/gdrive/My Drive/Workplace


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Jul 17 07:04:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import copy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class ReplayBuffer(object):
    def __init__(self, state_dim, action_dim, max_size=int(1e6)):
        self.max_size = max_size
        self.ptr = 0
        self.size = 0

        self.start_ptr = 0
        self.start_size = 0

        self.state = np.zeros((max_size, state_dim))
        self.action = np.zeros((max_size, action_dim))
        self.next_state = np.zeros((max_size, state_dim))
        self.reward = np.zeros((max_size, 1))
        self.not_done = np.zeros((max_size, 1))

        self.start_state = np.zeros((max_size, state_dim))

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def add(self, state, action, next_state, reward, done):
        self.state[self.ptr] = state
        self.action[self.ptr] = action
        self.next_state[self.ptr] = next_state
        self.reward[self.ptr] = reward
        self.not_done[self.ptr] = 1. - done

        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)

    def sample(self, batch_size):
        ind = np.random.randint(self.size, size=batch_size)

        return (
            torch.FloatTensor(self.state[ind]).to(self.device),
            torch.FloatTensor(self.action[ind]).to(self.device),
            torch.FloatTensor(self.next_state[ind]).to(self.device),
            torch.FloatTensor(self.reward[ind]).to(self.device),
            torch.FloatTensor(self.not_done[ind]).to(self.device)
        )


class Encoder_Decoder(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Encoder_Decoder, self).__init__()

        self.e1 = nn.Linear(state_dim + action_dim, 256)
        self.e2 = nn.Linear(256, 256)

        self.r1 = nn.Linear(256, 1, bias=False)

        self.a1 = nn.Linear(256, 256)
        self.a2 = nn.Linear(256, action_dim)

        self.d1 = nn.Linear(256, 256)
        self.d2 = nn.Linear(256, state_dim)


    def forward(self, state, action):
        l = F.relu(self.e1(torch.cat([state, action], 1)))
        l = F.relu(self.e2(l))
        # l is the latent representation

        # reward prediction
        r = self.r1(l)

        # next_state prediction
        d = F.relu(self.d1(l))
        ns = self.d2(d)

        # action reconstruction
        d = F.relu(self.a1(l))
        a = self.a2(d)

        return ns, r, a, l

    def latent(self, state, action):
        l = F.relu(self.e1(torch.cat([state, action], 1)))
        l = F.relu(self.e2(l))
        return l


class Critic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Critic, self).__init__()

        self.l1 = nn.Linear(state_dim + action_dim, 256)
        self.l2 = nn.Linear(256, 256)
        self.l3 = nn.Linear(256, 256)

    def forward(self, state, action):
        q1 = F.relu(self.l1(torch.cat([state, action], 1)))
        q1 = F.relu(self.l2(q1))
        return self.l3(q1)

# TODO : Try using double-q network
class DoubleCritic(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(DoubleCritic, self).__init__()

        self.l1 = nn.Linear(state_dim + action_dim, 256)
        self.l2 = nn.Linear(256, 256)
        self.l3 = nn.Linear(256, 256)

        self.r1 = nn.Linear(state_dim + action_dim, 256)
        self.r2 = nn.Linear(256, 256)
        self.r3 = nn.Linear(256, 256)

    def forward(self, state, action):
        q1 = F.relu(self.l1(torch.cat([state, action], 1)))
        q1 = F.relu(self.l2(q1))

        q2 = F.relu(self.r1(torch.cat([state, action], 1)))
        q2 = F.relu(self.r2(q2))
        return self.l3(q1), self.r3(q2)

    def Q1(self, state, action):
        q1 = F.relu(self.l1(torch.cat([state, action], 1)))
        q1 = F.relu(self.l2(q1))

        return self.l3(q1)
  

class Actor(nn.Module):
    def __init__(self, state_dim, action_dim, max_action):
        super(Actor, self).__init__()

        self.l1 = nn.Linear(state_dim, 256)
        self.l2 = nn.Linear(256, 256)
        self.l3 = nn.Linear(256, action_dim)
        
        self.max_action = max_action

    def forward(self, state):
        state = torch.tensor(state).to(device)
        a = F.relu(self.l1(state))
        a = F.relu(self.l2(a))
        return self.max_action * torch.tanh(self.l3(a))

class WNet(nn.Module):
    def __init__(self):
        super(WNet, self).__init__()
        self.fc = nn.Linear(256, 1, bias=False)
        
    
    def forward(self, latent):
        return self.fc(latent)


class DSFPG_doubleQ(object):
    def __init__(
        self,
        state_dim,
        action_dim,
        max_action,
        buffer_size=int(1e6),
        discount=0.99,
        tau=0.005,
        delay=2,
        policy_noise=0.2,
        noise_clip=0.5,
        preserve_SR=False,
    ):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.encoder_decoder = Encoder_Decoder(state_dim, action_dim).to(device)
        self.ed_optimizer = torch.optim.Adam(self.encoder_decoder.parameters(), lr=3e-4)

        self.actor = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target = copy.deepcopy(self.actor)
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=3e-4)

        self.critic = DoubleCritic(state_dim, action_dim).to(device)
        self.critic_target = copy.deepcopy(self.critic)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=3e-4)

        # self.W = torch.ones(1, 256, requires_grad=True, device=device)
        self.W = WNet().to(device)
        # self.W_optimizer = torch.optim.Adam([self.W], lr=3e-4)
        self.W_optimizer = torch.optim.Adam(self.W.parameters(), lr=3e-4)

        self.discount = discount
        self.tau = tau

        self.total_it = 0

        self.max_action = max_action

        self.replay = ReplayBuffer(state_dim, action_dim, buffer_size)

        # TD3 improvement
        self.delay = delay
        self.policy_noise = policy_noise
        self.noise_clip = noise_clip

        # perserve SR or not
        self.preserve_SR = preserve_SR

        # psi_loss check
        self.psi_loss = []

    def train_encoder_decoder(self, replay_buffer, batch_size=256):
        # self.replay.add(state, action, next_state, reward, done)
        state, action, next_state, reward, not_done = replay_buffer.sample(batch_size)

        recons_next, recons_reward, recons_action, lat = self.encoder_decoder(state, action)
        ed_loss = F.mse_loss(recons_next, next_state) + 0.1 * F.mse_loss(recons_reward, reward) + F.mse_loss(recons_action, action)

        self.ed_optimizer.zero_grad()
        ed_loss.backward()
        self.ed_optimizer.step()


    def train_SR(self, replay_buffer, batch_size=256):
        # self.replay.add(state, action, next_state, reward, done)
        # print('start training')
        state, action, next_state, reward, not_done = replay_buffer.sample(batch_size)

        with torch.no_grad():
            next_action = self.actor_target(next_state)
            # add randomness to the next action, this should be removed if the result is not ideal
            # next_action = (next_action + torch.randn_like(next_action) * self.max_action * 0.1).clamp(-self.max_action, self.max_action)

            latent = self.encoder_decoder.latent(state, action)
            target_Q = latent + self.discount * not_done * self.critic_target(next_state, next_action)

        current_Q = self.critic(state, action)
        critic_loss = F.mse_loss(current_Q, target_Q)

        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()

        self.psi_loss.append(critic_loss)

        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
            target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)


    def train_w(self, replay_buffer, batch_size=256):
        # self.replay.add(state, action, next_state, reward, done)
    
        state, action, next_state, reward, not_done = replay_buffer.sample(batch_size)

        with torch.no_grad():
            latent = self.encoder_decoder.latent(state, action)
        
        reward_estimate = self.W(latent)
        W_loss = F.mse_loss(reward_estimate, reward)

        self.W_optimizer.zero_grad()
        W_loss.backward()
        self.W_optimizer.step()

    def train_actor(self, replay_buffer, batch_size=256):
        # self.replay.add(state, action, next_state, reward, done)
        state, action, next_state, reward, not_done = replay_buffer.sample(batch_size)

        # actor_loss = -(self.critic(state, self.actor(state)) * self.W).mean()
        actor_loss = -self.W(self.critic(state, self.actor(state))).mean()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()


    def soft_update(self):
        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
                target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)


    def train_all(self, replay_buffer, batch_size=256):

        state, action, next_state, reward, not_done = replay_buffer.sample(batch_size)
        '''
        Train Encoder-Decoder
        '''
        recons_next, recons_reward, recons_action, lat = self.encoder_decoder(state, action)
        ed_loss = F.mse_loss(recons_next, next_state) + \
                    0.1 * F.mse_loss(recons_reward, reward) + \
                    F.mse_loss(recons_action, action)

        self.ed_optimizer.zero_grad()
        ed_loss.backward()
        self.ed_optimizer.step()

        '''
        Train SR
        '''
        if not self.perserve_SR: # if we don't have a mature SR yet, learn it
            with torch.no_grad():
                # next_action = self.actor_target(next_state) # try random action instead
                noise = (
                    torch.randn_like(action) * self.policy_noise
                ).clamp(-self.noise_clip, self.noise_clip)

                next_action = (
                    self.actor_target(next_state) + noise
                    # noise / self.policy_noise
                ).clamp(-self.max_action, self.max_action)
                # add randomness to the next action, this should be removed if the result is not ideal
                # next_action = (next_action + torch.randn_like(next_action) * self.max_action * 0.1).clamp(-self.max_action, self.max_action)
                latent = self.encoder_decoder.latent(state, action)
                target_Q1, target_Q2 = self.critic_target(next_state, next_action) # target_Q's are of shape (256, 256)
                target_value1, target_value2 = self.W(target_Q1), self.W(target_Q2)
                
                min_q = torch.where(target_value1 < target_value2, target_Q1, target_Q2) # pick the sf's leading to minimum q-value
                # target_Q = latent + self.discount * not_done * self.critic_target(next_state, next_action)
                target_Q = latent + self.discount * not_done * min_q

            # need to change to double-Q function
            # TODO: Add min-q trick / mean-q update
            current_Q1, current_Q2 = self.critic(state, action)
            
            # compute the critic loss
            critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)

            self.critic_optimizer.zero_grad()
            critic_loss.backward()
            self.critic_optimizer.step()

        '''
        Train W
        '''
        with torch.no_grad():
            latent = self.encoder_decoder.latent(state, action)
        
        reward_estimate = self.W(latent)
        W_loss = F.mse_loss(reward_estimate, reward)

        self.W_optimizer.zero_grad()
        W_loss.backward()
        self.W_optimizer.step()

        # use delayed update instead
        # self.soft_update()

        # TODO : Add delayed update
        '''
        Train actors
        '''
        # actor_loss = -(self.critic(state, self.actor(state)) * self.W).mean()
        actor_loss = -self.W(self.critic.Q1(state, self.actor(state))).mean()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        self.soft_update()

        # if self.total_it % self.delay == 0:
        #     '''
        #     Train actors
        #     '''
        #     # actor_loss = -(self.critic(state, self.actor(state)) * self.W).mean()
        #     actor_loss = -self.W(self.critic.Q1(state, self.actor(state))).mean()
        #     self.actor_optimizer.zero_grad()
        #     actor_loss.backward()
        #     self.actor_optimizer.step()

        #     self.soft_update()

    def select_action(self, state):
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        return self.actor(state).cpu().data.numpy().flatten()


    def train(self, replay_buffer, batch_size=256):
        self.total_it += 1
        self.train_all(replay_buffer)


        # self.train_actor(state, action, next_state, reward, done)
        # self.train_encoder_decoder(state, action, next_state, reward, done)
        # self.train_SR(state, action, next_state, reward, done)
        # self.train_w(state, action, next_state, reward, done)

In [ ]:
class PsiNet(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(PsiNet, self).__init__()
        self.l1 = nn.Linear(state_dim + action_dim, 256)
        self.l2 = nn.Linear(256, 256)
        self.l3 = nn.Linear(256, state_dim)

        self.r1 = nn.Linear(state_dim + action_dim, 256)
        self.r2 = nn.Linear(256, 256)
        self.r3 = nn.Linear(256, state_dim)

    def forward(self, state, action):
        cat = torch.cat([state, action], axis=1)

        x = F.relu(self.l1(cat))
        x = F.relu(self.l2(x))

        y = F.relu(self.r1(cat))
        y = F.relu(self.r2(y))

        return self.l3(x), self.r3(y)

    def Q1(self, state, action):
        cat = torch.cat([state, action], axis=1)

        x = F.relu(self.l1(cat))
        x = F.relu(self.l2(x))

        return self.l3(x)

class RewardNet(nn.Module):
    def __init__(self, state_dim):
        super(RewardNet, self).__init__()
        self.fc1 = nn.Linear(state_dim, 1)
    
    def forward(self, state):
        return self.fc1(state)

class DSFPG(object):
    def __init__(
        self,
        state_dim,
        action_dim,
        max_action,
        buffer_size=int(1e6),
        discount=0.99,
        tau=0.005,
        delay=2,
        policy_noise=0.2,
        noise_clip=0.5,
        preserve_SR=False,
    ):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.actor = Actor(state_dim, action_dim, max_action).to(device)
        self.actor_target = copy.deepcopy(self.actor)
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(), lr=3e-4)

        self.critic = PsiNet(state_dim, action_dim).to(device)
        self.critic_target = copy.deepcopy(self.critic)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(), lr=3e-4)

        # self.W = torch.ones(1, 256, requires_grad=True, device=device)
        self.W = RewardNet(state_dim).to(device)
        # self.W_optimizer = torch.optim.Adam([self.W], lr=3e-4)
        self.W_optimizer = torch.optim.Adam(self.W.parameters(), lr=3e-4)

        self.discount = discount
        self.tau = tau

        self.total_it = 0

        self.max_action = max_action

        self.replay = ReplayBuffer(state_dim, action_dim, buffer_size)

        # TD3 improvement
        self.delay = delay
        self.policy_noise = policy_noise
        self.noise_clip = noise_clip

        # perserve SR or not
        self.preserve_SR = preserve_SR

        # psi_loss check
        self.psi_loss = []

    def soft_update(self):
        for param, target_param in zip(self.critic.parameters(), self.critic_target.parameters()):
                target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)

        for param, target_param in zip(self.actor.parameters(), self.actor_target.parameters()):
                target_param.data.copy_(self.tau * param.data + (1 - self.tau) * target_param.data)


    def train_all(self, replay_buffer, batch_size=256):

        state, action, next_state, reward, not_done = replay_buffer.sample(batch_size)

        '''
        Train SR
        '''
        if not self.perserve_SR: # if we don't have a mature SR yet, learn it
            with torch.no_grad():
                # next_action = self.actor_target(next_state) # try random action instead
                noise = (
                    torch.randn_like(action) * self.policy_noise
                ).clamp(-self.noise_clip, self.noise_clip)

                next_action = (
                    self.actor_target(next_state) + noise
                    # noise / self.policy_noise
                ).clamp(-self.max_action, self.max_action)
                # add randomness to the next action, this should be removed if the result is not ideal
                # next_action = (next_action + torch.randn_like(next_action) * self.max_action * 0.1).clamp(-self.max_action, self.max_action)
                # latent = self.encoder_decoder.latent(state, action)
                target_Q1, target_Q2 = self.critic_target(next_state, next_action) # target_Q's are of shape (256, 256)
                target_value1, target_value2 = self.W(target_Q1), self.W(target_Q2)
                
                min_q = torch.where(target_value1 < target_value2, target_Q1, target_Q2) # pick the sf's leading to minimum q-value
                # target_Q = latent + self.discount * not_done * self.critic_target(next_state, next_action)
                target_Q = state + self.discount * not_done * min_q

            # need to change to double-Q function
            # TODO: Add min-q trick / mean-q update
            current_Q1, current_Q2 = self.critic(state, action)
            
            # compute the critic loss
            critic_loss = F.mse_loss(current_Q1, target_Q) + F.mse_loss(current_Q2, target_Q)

            self.critic_optimizer.zero_grad()
            critic_loss.backward()
            self.critic_optimizer.step()

        '''
        Train W
        '''
        reward_estimate = self.W(next_state)
        W_loss = F.mse_loss(reward_estimate, reward)

        self.W_optimizer.zero_grad()
        W_loss.backward()
        self.W_optimizer.step()

        # use delayed update instead
        # self.soft_update()

        # TODO : Add delayed update
        actor_loss = -self.W(self.critic.Q1(state, self.actor(state))).mean()
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()

        self.soft_update()

        # if self.total_it % self.delay == 0:
        #     '''
        #     Train actors
        #     '''
        #     # actor_loss = -(self.critic(state, self.actor(state)) * self.W).mean()
        #     actor_loss = -self.W(self.critic.Q1(state, self.actor(state))).mean()
        #     self.actor_optimizer.zero_grad()
        #     actor_loss.backward()
        #     self.actor_optimizer.step()

        #     self.soft_update()

    def select_action(self, state):
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        return self.actor(state).cpu().data.numpy().flatten()


    def train(self, replay_buffer, batch_size=256):
        self.total_it += 1
        self.train_all(replay_buffer)

In [ ]:
from tqdm import trange
import gym
import pybulletgym

def eval_policy(policy, env, seed, eval_episodes=10):
    if isinstance(env, str):
        eval_env = gym.make(env)
    else:
        eval_env = env # a non_proper way 
    eval_env.seed(seed + 100)

    avg_reward = 0.
    for _ in range(eval_episodes):
        state, done = eval_env.reset(), False
        while not done:
            action = policy.select_action(np.array(state))
            state, reward, done, _ = eval_env.step(action)
            avg_reward += reward

    avg_reward /= eval_episodes

    print("---------------------------------------")
    print(f"Evaluation over {eval_episodes} episodes: {avg_reward:.3f}")
    print("---------------------------------------")
    return avg_reward

def train_agent(
    agent_func,
    env_name,
    max_steps=int(2e5),
    max_step_before_learning=10000,
    preserve_SR=False,
):
    if isinstance(env_name, str):
        env = gym.make(env_name)
    else:
        env = env_name
    state_dim = env.observation_space.shape[0]
    action_dim = env.action_space.shape[0]
    max_action = float(env.action_space.high[0])

    replay_buffer = ReplayBuffer(state_dim, action_dim)

    policy = agent_func(state_dim, action_dim, max_action) if callable(agent_func) else agent_func # if agent_func is a function, create an agent, otherwise, use the agent
    policy.perserve_SR = preserve_SR

    evaluations = [eval_policy(policy, env_name, 0)] # 0 is the seed

    state, done = env.reset(), False
    episode_reward = 0
    episode_timesteps = 0
    episode_num = 0

    for t in range(int(max_steps)):
        
        episode_timesteps += 1

        # Select action randomly or according to policy
        if t < max_step_before_learning:
            action = env.action_space.sample()
        else:
            action = (
                policy.select_action(np.array(state))
                + np.random.normal(0, max_action * 0.2, size=action_dim)
            ).clip(-max_action, max_action)

        # Perform action
        next_state, reward, done, _ = env.step(action) 
        done_bool = float(done) if episode_timesteps < env._max_episode_steps else 0

        # Store data in replay buffer
        replay_buffer.add(state, action, next_state, reward, done_bool)

        state = next_state
        episode_reward += reward

        # Train agent after collecting sufficient data
        if t >= max_step_before_learning:
            policy.train(replay_buffer, 128)

        if done: 
            # +1 to account for 0 indexing. +0 on ep_timesteps since it will increment +1 even if done=True
            print(f"Total T: {t+1} Episode Num: {episode_num+1} Episode T: {episode_timesteps} Reward: {episode_reward:.3f}")
            # Reset environment
            state, done = env.reset(), False
            episode_reward = 0
            episode_timesteps = 0
            episode_num += 1 

        # Evaluate episode
        if (t + 1) % 1000 == 0:
            evaluations.append(eval_policy(policy, env_name, 0))
    
    return evaluations, policy

In [ ]:
evaluations, policy = train_agent(DSFPG, 'HopperMuJoCoEnv-v0')

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


WalkerBase::__init__
WalkerBase::__init__
---------------------------------------
Evaluation over 10 episodes: 4.380
---------------------------------------
Total T: 3 Episode Num: 1 Episode T: 3 Reward: 1.822
Total T: 9 Episode Num: 2 Episode T: 6 Reward: 5.208
Total T: 13 Episode Num: 3 Episode T: 4 Reward: 2.646
Total T: 18 Episode Num: 4 Episode T: 5 Reward: 3.501
Total T: 24 Episode Num: 5 Episode T: 6 Reward: 2.530
Total T: 26 Episode Num: 6 Episode T: 2 Reward: 1.589
Total T: 46 Episode Num: 7 Episode T: 20 Reward: 20.827
Total T: 50 Episode Num: 8 Episode T: 4 Reward: 3.003
Total T: 54 Episode Num: 9 Episode T: 4 Reward: 4.271
Total T: 56 Episode Num: 10 Episode T: 2 Reward: 1.693
Total T: 60 Episode Num: 11 Episode T: 4 Reward: 2.194
Total T: 62 Episode Num: 12 Episode T: 2 Reward: 1.548
Total T: 66 Episode Num: 13 Episode T: 4 Reward: 2.863
Total T: 71 Episode Num: 14 Episode T: 5 Reward: 3.985
Total T: 79 Episode Num: 15 Episode T: 8 Reward: 7.211
Total T: 88 Episode Num: 16

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:142: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Total T: 95 Episode Num: 17 Episode T: 7 Reward: 9.017
Total T: 97 Episode Num: 18 Episode T: 2 Reward: 0.377
Total T: 102 Episode Num: 19 Episode T: 5 Reward: 3.189
Total T: 106 Episode Num: 20 Episode T: 4 Reward: 2.908
Total T: 108 Episode Num: 21 Episode T: 2 Reward: 1.758
Total T: 117 Episode Num: 22 Episode T: 9 Reward: 5.956
Total T: 120 Episode Num: 23 Episode T: 3 Reward: 2.651
Total T: 130 Episode Num: 24 Episode T: 10 Reward: 6.996
Total T: 144 Episode Num: 25 Episode T: 14 Reward: 14.167
Total T: 151 Episode Num: 26 Episode T: 7 Reward: 4.163
Total T: 155 Episode Num: 27 Episode T: 4 Reward: 0.758
Total T: 161 Episode Num: 28 Episode T: 6 Reward: 6.347
Total T: 164 Episode Num: 29 Episode T: 3 Reward: 1.939
Total T: 170 Episode Num: 30 Episode T: 6 Reward: 3.740
Total T: 173 Episode Num: 31 Episode T: 3 Reward: 1.585
Total T: 175 Episode Num: 32 Episode T: 2 Reward: 2.079
Total T: 179 Episode Num: 33 Episode T: 4 Reward: 2.500
Total T: 181 Episode Num: 34 Episode T: 2 Rewar

In [ ]:
evaluations, policy = train_agent(DSFPG_doubleQ, 'HopperMuJoCoEnv-v0')

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


WalkerBase::__init__
WalkerBase::__init__


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:142: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


---------------------------------------
Evaluation over 10 episodes: 21.982
---------------------------------------
Total T: 5 Episode Num: 1 Episode T: 5 Reward: 4.019
Total T: 7 Episode Num: 2 Episode T: 2 Reward: 1.622
Total T: 10 Episode Num: 3 Episode T: 3 Reward: 3.231
Total T: 15 Episode Num: 4 Episode T: 5 Reward: 3.777
Total T: 25 Episode Num: 5 Episode T: 10 Reward: 5.412
Total T: 31 Episode Num: 6 Episode T: 6 Reward: 3.311
Total T: 37 Episode Num: 7 Episode T: 6 Reward: 5.069
Total T: 46 Episode Num: 8 Episode T: 9 Reward: 8.967
Total T: 54 Episode Num: 9 Episode T: 8 Reward: 6.582
Total T: 57 Episode Num: 10 Episode T: 3 Reward: -1.915
Total T: 61 Episode Num: 11 Episode T: 4 Reward: 2.883
Total T: 64 Episode Num: 12 Episode T: 3 Reward: 2.274
Total T: 69 Episode Num: 13 Episode T: 5 Reward: 5.645
Total T: 80 Episode Num: 14 Episode T: 11 Reward: 3.536
Total T: 82 Episode Num: 15 Episode T: 2 Reward: 1.398
Total T: 84 Episode Num: 16 Episode T: 2 Reward: 1.739
Total T: 87 

KeyboardInterrupt: ignored